In [287]:
#import dependencies
import numpy as np
import pandas as pd
import tweepy
import time
import json
import datetime as dt
import csv
from pprint import pprint
from matplotlib import pyplot as plt
from matplotlib import patches as mpatches
import seaborn as sns
from config import consumer_key, consumer_secret, access_token, access_token_secret
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
# Twitter API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret

In [288]:
# Twitter Credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [289]:
def TwitterBot():
    acquiretarget()
    searchfortweets(target_sn)
    analyzeandplot(searchfortweets())
    return print('Hi')

def search_for_tweets(target_sn):
    tweets_ago = 0
    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    tweet_num_list = []

    # Loop through 25 times
    for x in range(2):

        # Pull a page of tweets from the target_sn's timeline
        public_tweets = api.user_timeline(target_sn, page=x, result_type="recent")

        # Loop through all tweets
        for tweet in public_tweets:

            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]

            # Add each value to the appropriate list
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)
            
            #Increment the num_tweets to count the tweets from most recent
            tweet_num_list.append(tweets_ago)
            tweets_ago -= 1
                
    # return the lists and target_sn for next method
    return compound_list,positive_list,negative_list,neutral_list,tweet_num_list,target_sn

def analyze_and_plot(compound_list,positive_list,negative_list,neutral_list,tweet_num_list,target_sn):
    # Begin by constructing dataframe from tweet polarity lists
    tweetdf = pd.DataFrame(compound_list,columns=['Compound Score'])
    tweetdf['Positive Score'] = positive_list
    tweetdf['Negative Score'] = negative_list
    tweetdf['Neutral Score'] = neutral_list
    # Rename index to tweets ago value
    tweetdf.set_axis(tweet_num_list,axis=0,inplace=True)
    # Define Seaborn style and generate plot
    sns.set_style('darkgrid')
    tweetplot = sns.tsplot(data=tweetdf['Compound Score'],time=tweetdf.index.values,condition=['Compound Score'])
    # Label x and y axes
    tweetplot.set_ylabel('Tweet Polarity')
    tweetplot.set_xlabel('Tweets Ago')
    # Create a patch for the legend to reflect the target_sn
    user = mpatches.Patch(label=target_sn)
    tweetplot.legend(handles=[user])
    # Save the file to post later
    plt.savefig((f'{target_sn}'),dpi=300)
    return target_sn

In [324]:
def identify_check_request():
    #Declare local variables
    recent_sn = ''
    requester = ''
    requesting_id = ''
    list_of_targets = []
    # Look at the most recent tweet on the bot's timeline and extract text content and author's sn
    request_tweet = api.user_timeline('@AwayMikes',count=1,result_type='recent')
    request_text = request_tweet[0]['text']
    requester = '@'+request_tweet[0]['user']['screen_name']
    requesting_id = request_tweet[0]['id_str']
    #pprint(request_tweet)
    # For simplicity sake, extract the last element that was split on @
    # presumably this will be the target SN, if it is not, it will be handled later
    _sn_start = request_text.rfind('@')
    at_to_end = request_text[_sn_start:]
    split_on_space = at_to_end.split(' ')
    #recent_sn = split_on_space[0]
    recent_sn = 'test1'
    with open ('list_of_targets.csv',newline='') as csvfile:
        target_reader = csv.reader(csvfile,delimiter=',')
        for row in target_reader:
            list_of_targets.append(row)
    if recent_sn not in list_of_targets:
        print("Something ain't right")
        print(recent_sn)
        print(list_of_targets)
    if recent_sn in list_of_targets:
        print("BOYHOWDY I FOUND THIS ONE AGAIN!")
        api.update_with_media(f"I'm sorry {requester}, {recent_sn} has already been analyzed. Here is the plot: ",
                              requesting_id)

In [325]:
identify_check_request()

Something ain't right
test1
[['@realDonaldTrump', 'test1']]


In [ ]:
analyzeandplot(*searchfortweets('@RealDonaldTrump'))